In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk.classify.util as util
import itertools
import pickle
from nltk.probability import FreqDist
import os.path
from statistics import mode
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.probability import FreqDist
import numpy as np
import re
import math
import pandas as pd
import csv
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
import numpy as np


nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
emoji_dict = {}
with open('emoji.txt', 'r', encoding='latin-1') as emoji_file:
    for line in emoji_file:
        line = line.strip()
        if line:
            emoji, value = line.split('\t')
            emoji_dict[emoji] = int(value)

In [13]:
def replace_emojis(text, emoji_dict):
    for emoji, value in emoji_dict.items():
        if value == 1:
            text = re.sub(re.escape(emoji), 'happy', text)
        elif value == -1:
            text = re.sub(re.escape(emoji), 'sad', text)
    return text

In [14]:
#vincent
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Tokenize the words
    words = word_tokenize(text)
    
    # Apply stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tagged = pos_tag(words)
    words = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)) if get_wordnet_pos(pos) else word for word, pos in tagged]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    return ' '.join(filtered_words)

hatefulCorpus=[]
nHatefulCorpus=[]
neutralCorpus=[]
with open("firstIter.csv", "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        if not all(value == "" for value in row.values()):
            text = row["Comment Text"]
            newText = text.strip()
            newText = replace_emojis(newText, emoji_dict)
            newText = preprocess_text(newText)
            label = row["Label"]
            if label=='Neutral':
                neutralCorpus.append(newText)
            elif label=='Hateful':
                hatefulCorpus.append(newText)
            else:
                nHatefulCorpus.append(newText)

In [15]:
print(str(len(neutralCorpus))+" "+str(len(hatefulCorpus))+" "+str(len(nHatefulCorpus)))

100 437 1976


In [16]:

all_texts = hatefulCorpus + nHatefulCorpus + neutralCorpus
# Create frequency distribution
word_freq = FreqDist()

for text in all_texts:
    words = text.split()
    word_freq.update(words)
known_words = {word for word, freq in word_freq.items() if freq > 3}
len(known_words)

1547

In [17]:
total_words = 0
num_comments = len(all_texts)

for comment in all_texts:
    total_words += len(comment.split())

average_words = total_words / num_comments

variance = 0
for comment in all_texts:
    num_words = len(comment.split())
    variance += (num_words - average_words) ** 2

std_deviation = math.sqrt(variance / num_comments)
M = round(average_words + std_deviation)

def comment_to_vector(comment, known_words, M):
    words = comment.split()
    vector = []
    for i in range(M):
        if i < len(words):
            word = words[i]
            if word in known_words:
                vector.append(known_words.index(word) + 1)
            else:
                vector.append(0)
        else:
            vector.append(0)
    return vector

# Convert preprocessed comments to vector representation
hateful_comment_vectors = []
for comment in hatefulCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    hateful_comment_vectors.append(vector)
    

nHateful_comment_vectors = []
for comment in nHatefulCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    nHateful_comment_vectors.append(vector)
    
neutral_comment_vectors = []
for comment in neutralCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    neutral_comment_vectors.append(vector)

In [18]:
len(hateful_comment_vectors[6])

40

In [19]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError("embedding dimension = %d should be divisible by number of heads = %d" % (embed_dim, num_heads))
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


embed_dim = 32  # Embedding size for each


In [23]:
from tensorflow.keras.utils import to_categorical
class TransformerClassifier(tf.keras.Model):
    def __init__(self, vocab_size, maxlen, embed_dim, num_heads, ff_dim):
        super(TransformerClassifier, self).__init__()
        self.embedding = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout = Dropout(0.5)
        self.pool = layers.GlobalAveragePooling1D()
        self.classifier = layers.Dense(num_classes, activation='softmax', 
                               kernel_regularizer=regularizers.l2(0.01))

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.transformer_block(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

# Parameters for the model
vocab_size = 20000  
maxlen = 200  
embed_dim = 32  
num_heads = 2  
ff_dim = 32  

hateful_comment_vectors = np.array(hateful_comment_vectors)
nHateful_comment_vectors = np.array(nHateful_comment_vectors)
neutral_comment_vectors = np.array(neutral_comment_vectors)

all_comment_vectors = np.concatenate((hateful_comment_vectors, nHateful_comment_vectors, neutral_comment_vectors))

hateful_labels = np.ones(len(hatefulCorpus))
nHateful_labels = np.zeros(len(nHatefulCorpus))
neutral_labels = np.full(len(neutralCorpus), 2)

all_labels = np.concatenate((hateful_labels, nHateful_labels, neutral_labels))
num_classes=3

X_train, X_test, y_train, y_test = train_test_split(all_comment_vectors, all_labels, test_size=0.2, random_state=42)
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)

# Initialize the model
model = TransformerClassifier(vocab_size, maxlen, embed_dim, num_heads, ff_dim)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Train the model
history = model.fit(
    X_train, y_train_one_hot, 
    epochs=10, 
    batch_size=32, 
    validation_data=(X_test, y_test_one_hot), 
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

model.save('savedModels/Transformer_Model')

Epoch 1/10
63/63 [==============================] - 4s 24ms/step - loss: 0.7616 - accuracy: 0.7572 - val_loss: 0.6190 - val_accuracy: 0.8091
Epoch 2/10
63/63 [==============================] - 1s 19ms/step - loss: 0.6569 - accuracy: 0.7841 - val_loss: 0.6053 - val_accuracy: 0.8091
Epoch 3/10
63/63 [==============================] - 1s 20ms/step - loss: 0.6030 - accuracy: 0.8000 - val_loss: 0.7172 - val_accuracy: 0.7475
Epoch 4/10
16/16 [==============================] - 0s 4ms/step - loss: 0.6259 - accuracy: 0.7972
Test Loss: 0.6259426474571228
Test Accuracy: 0.7972167134284973


INFO:tensorflow:Assets written to: savedModels/Transformer_Model\assets


INFO:tensorflow:Assets written to: savedModels/Transformer_Model\assets


In [24]:
import tensorflow as tf
import keras
testData = "All immigrants should be deported to where they came from."
testData = testData.strip()
testData = replace_emojis(testData, emoji_dict)
testData = preprocess_text(testData)
testDataVector = comment_to_vector(testData, list(known_words), 40)
testDataVector = np.array(testDataVector)
testDataVector = np.expand_dims(testDataVector, axis=0)
loadedModel = tf.keras.models.load_model('savedModels/LSTM_Model')
predicted_probabilities = loadedModel.predict(testDataVector)
predicted_class = np.argmax(predicted_probabilities)
class_labels = ['hateful', 'non-hateful', 'neutral']
print(class_labels[predicted_class])

1/1 [==============================] - 1s 669ms/step
hateful
